# 空氣污染監測網 網路爬蟲實作練習


* 能夠利用 selenium + BeautifulSoup 撰寫爬蟲，並存放到合適的資料結構


## 作業目標

根據範例 ，完成以下問題：

* ① 取出 台北市士林區 2018/01 – 2018/08 的 SO2 資料
* ② 取出 台北市士林區 2018/01 – 2018/08 的 SO2、CO 資料





In [1]:
import time
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import pandas as pd

browser = webdriver.Firefox()
browser.get("http://taqm.epa.gov.tw/taqm/tw/MonthlyAverage.aspx")
selectSite = Select(browser.find_element_by_id("ctl05_ddlSite"))
selectSite.select_by_value('11')
selectYear = Select(browser.find_element_by_id("ctl05_ddlYear"))
selectYear.select_by_value('2018')
browser.find_element_by_id('ctl05_btnQuery').click()
time.sleep(3)
html_source = browser.page_source
browser.close()
soup = BeautifulSoup(html_source, 'html.parser')
table = soup.find('table', class_='TABLE_G')
d = {}
for tr in table.find_all('tr')[1:]:
    for i, td in enumerate(tr.find_all('td')):
        if len(tr.find_all('td')) == 5:
            if i == 0:
                obs = td.text
                d.setdefault(obs, {})
            elif i == 2:
                date = td.text
            elif i == 3:
                value = td.text
                d[obs][date] = value
        
        elif len(tr.find_all('td')) == 3:
            if i == 0:
                date = td.text
            elif i == 1:
                value = td.text
                d[obs][date] = value


### ① 取出 台北市士林區 2018/01 – 2018/08 的 SO2 資料

In [2]:
d_SO2 = [{'時間': key, 'SO2': value} for key, value in d['SO2'].items() if key < '2018/09' ]
pd.DataFrame(d_SO2)

,時間,SO2
0,2018/01,1.80
1,2018/02,1.90
2,2018/03,2.20
3,2018/04,2.30
4,2018/05,3.10
5,2018/06,2.70
6,2018/07,2.20
7,2018/08,2.40


### ② 取出 台北市士林區 2018/01 – 2018/08 的 SO2、CO 資料

In [3]:
d_CO = [{'時間': key, 'CO': value} for key, value in d['CO'].items() if key < '2018/09' ]
pd.merge(pd.DataFrame(d_SO2), pd.DataFrame(d_CO))

,時間,SO2,CO
0,2018/01,1.80,0.34
1,2018/02,1.90,0.44
2,2018/03,2.20,0.40
3,2018/04,2.30,0.38
4,2018/05,3.10,0.34
5,2018/06,2.70,0.29
6,2018/07,2.20,0.21
7,2018/08,2.40,0.30
